# Load Store Handler Class

In [1]:
from linkingtool.hdf5_handler import DataHandler
import linkingtool.visuals as vis

In [2]:
import warnings
# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Define Province Code

In [ ]:
province_code:str='BC' # The tool is designed to work for any province of CANADA.
store=f"../data/store/resources_{province_code}.h5" 
datahandler=DataHandler(store) # the DataHandler object could be initiated without the store definition as well.

# What's inside the Store ?

In [ ]:
datahandler.show_tree(store) # Shows the hierarchical data inside the store

# Load Data from Store

In [9]:
# Loading Geodataframes
cells=datahandler.from_store('cells')
boundary=datahandler.from_store('boundary')
timeseries_clusters_solar=datahandler.from_store('timeseries/clusters/solar')
timeseries_clusters_wind=datahandler.from_store('timeseries/clusters/wind')
# clusters_solar=datahandler.from_store('clusters/solar')
clusters_wind=datahandler.from_store('clusters/wind')

In [ ]:
clusters_wind

# Static Data Visuals in Interactive Maps

In [ ]:
import hvplot.pandas
import holoviews as hv
from holoviews import opts
from bokeh.layouts import gridplot
from bokeh.io import show

# Initialize Holoviews extension
hv.extension('bokeh')


# Define a dictionary to map columns to specific colormaps
cmap_mapping = {
    'lcoe_wind': 'cool',
    'potential_capacity_wind': 'Blues',
    'lcoe_solar': 'autumn',
    'CF_IEC2': 'RdYlGn',
    'wind_CF_mean': 'RdYlGn',
    'windspeed_ERA5': 'winter',
    'nearest_station_distance_km': 'Oranges',
    'potential_capacity_wind': 'Blues',
    'potential_capacity_solar': 'Oranges',
}

# Define a function to create individual plots
def create_plot(column_name, cmap):
    return cells.hvplot(
        color=column_name,
        cmap=cmap,
        geo=True,
        tiles='CartoDark',  # Default base map
        frame_width=300,  # Adjust the size of the plots
        frame_height=300,  # Adjust the size of the plots
        data_aspect=.5,
        alpha=0.8,
        line_color='None',
        line_width=0.1,
        hover_line_color='red'
    ).opts(title=column_name,
            show_grid=True,
            show_legend=True,
            tools=['hover', 'pan', 'wheel_zoom','reset','box_select'],
            legend_position='top_right'
        )

# Create a list of plots for each column
plots = [create_plot(col, cmap) for col, cmap in cmap_mapping.items()]

# Create a grid layout for the plots
grid = hv.Layout(plots).cols(3)  # Adjust the number of columns as needed

# Show the layout
hv.save(grid, '../docs/grid_plots.html')  # Save the grid layout as an HTML file



# Render the layout as a Bokeh object
bokeh_layout = hv.render(grid, backend='bokeh')

# Show the layout
show(bokeh_layout)

In [ ]:
from bokeh.io import show
import holoviews as hv

# Render the layout as a Bokeh object
bokeh_layout = hv.render(grid, backend='bokeh')

# Show the layout
show(bokeh_layout)


# Timeseries Plots

In [ ]:
clusters_solar=datahandler.from_store('clusters/solar')

In [ ]:
clusters_solar.columns

In [ ]:
clusters_solar

In [ ]:
timeseries_clusters_solar

In [ ]:
import pandas as pd
import hvplot.pandas
import panel as pn
import random

# Initialize Panel with the dark theme
pn.extension(theme='default')

# Load your DataFrames
df_solar = timeseries_clusters_solar  # Your solar DataFrame
df_wind = timeseries_clusters_wind    # Your wind DataFrame

# Create a list of the column names for the dropdowns
solar_options = df_solar.columns.tolist()
wind_options = df_wind.columns.tolist()

# Function to generate a random vibrant color
def get_random_vibrant_color():
    return "#{:02x}{:02x}{:02x}".format(random.randint(150, 255), random.randint(150, 255), random.randint(150, 255))


# Create a function to update the solar plot based on the selected time series
def update_solar_plot(selected_series):
    return df_solar[selected_series].hvplot.line(
        title=f"Time Series - Solar: {selected_series}",
        xlabel="DateTime",
        ylabel="Value",
        legend='top_left',
        width=1000,  # Width of the plot
        height=200,  # Height of the plot
        tools=['hover'],  # Enable hover tool
        line_color=get_random_vibrant_color()  # Random vibrant color for the solar plot
    )

# Create a function to update the wind plot based on the selected time series
def update_wind_plot(selected_series):
    return df_wind[selected_series].hvplot.line(
        title=f"Time Series - Wind: {selected_series}",
        xlabel="DateTime",
        ylabel="Value",
        legend='top_left',
        width=1000,  # Width of the plot
        height=200,  # Height of the plot
        tools=['hover'],  # Enable hover tool
        line_color=get_random_vibrant_color()  # Random vibrant color for the wind plot
    )

# Create dropdown widgets for selecting the time series
solar_dropdown = pn.widgets.Select(name='Select Solar Time Series', options=solar_options)
wind_dropdown = pn.widgets.Select(name='Select Wind Time Series', options=wind_options)

# Create dynamic panels that update the plot based on the dropdown selections
dynamic_solar_plot = pn.bind(update_solar_plot, selected_series=solar_dropdown)
dynamic_wind_plot = pn.bind(update_wind_plot, selected_series=wind_dropdown)

# Create a layout with the dropdowns and the plots in a two-row grid
grid_layout = pn.Column(
    solar_dropdown, dynamic_solar_plot,  # Solar plot in the first row
    wind_dropdown, dynamic_wind_plot     # Wind plot in the second row
)

# Save the grid layout as an HTML file
grid_layout.save('../docs/time_series_plots.html')

# Display the panel in a notebook or in a web application
grid_layout.show()

In [ ]:
clusters_solar.loc['CowichanValley_1']

In [ ]:
import pandas as pd
import hvplot.pandas
import panel as pn
import random

# Initialize Panel with the dark theme
pn.extension(theme='default')

# Load your DataFrames
df_solar = timeseries_clusters_solar  # Your solar DataFrame
df_wind = timeseries_clusters_wind    # Your wind DataFrame
clusters_solar = clusters_solar       # Your clusters_solar DataFrame

# Create a list of the column names for the dropdowns
solar_options = df_solar.columns.tolist()
wind_options = df_wind.columns.tolist()

# Function to generate a random vibrant color
def get_random_vibrant_color():
    return "#{:02x}{:02x}{:02x}".format(random.randint(150, 255), random.randint(150, 255), random.randint(150, 255))

# Create a function to update the solar plot based on the selected time series
def update_solar_plot(selected_series):
    return df_solar[selected_series].hvplot.line(
        title=f"Time Series - Solar: {selected_series}",
        xlabel="DateTime",
        ylabel="Value",
        legend='top_left',
        width=1000,  # Width of the plot
        height=300,  # Increased height for better visibility
        tools=['hover'],  # Enable hover tool
        line_color=get_random_vibrant_color(),  # Random vibrant color for the solar plot
        line_width=2,  # Make line thicker for better visibility
        fontsize={'title': 16, 'xlabel': 14, 'ylabel': 14, 'legend': 12},  # Adjust font sizes
    )

# Create a function to update the wind plot based on the selected time series
def update_wind_plot(selected_series):
    return df_wind[selected_series].hvplot.line(
        title=f"Time Series - Wind: {selected_series}",
        xlabel="DateTime",
        ylabel="Value",
        legend='top_left',
        width=1000,  # Width of the plot
        height=300,  # Increased height for better visibility
        tools=['hover'],  # Enable hover tool
        line_color=get_random_vibrant_color(),  # Random vibrant color for the wind plot
        line_width=2,  # Make line thicker for better visibility
        fontsize={'title': 16, 'xlabel': 14, 'ylabel': 14, 'legend': 12},  # Adjust font sizes
    )

# Create dropdown widgets for selecting the time series
solar_dropdown = pn.widgets.Select(name='Select Solar Time Series', options=solar_options, width=300)
wind_dropdown = pn.widgets.Select(name='Select Wind Time Series', options=wind_options, width=300)


# Create dynamic panels that update the plot based on the dropdown selections
dynamic_solar_plot = pn.bind(update_solar_plot, selected_series=solar_dropdown)
dynamic_wind_plot = pn.bind(update_wind_plot, selected_series=wind_dropdown)

# Function to get relevant row data from clusters_solar
def get_cluster_info(selected_series):
    # Use the selected series name to find the relevant row in clusters_solar
    selected_row = clusters_solar.loc[selected_series]
    return selected_row

# Create a function to update the table based on the selected time series
def update_cluster_table(selected_series):
    cluster_info = get_cluster_info(selected_series)
    
    # Drop the geometry column if it exists
    if 'geometry' in cluster_info.index:
        cluster_info = cluster_info.drop('geometry')
    
    # Return the DataFrame widget without the geometry column
    return pn.widgets.DataFrame(cluster_info.to_frame().T, width=800, height=200)


# Create dynamic panels for the table
dynamic_cluster_table = pn.bind(update_cluster_table, selected_series=solar_dropdown)

# Adjust the overall layout for better alignment and spacing
grid_layout = pn.Column(
    pn.Row(solar_dropdown, dynamic_solar_plot, align="center"),  # Center align solar dropdown and plot
    pn.Row(dynamic_cluster_table),  # Show the table below the solar plot
    pn.Row(wind_dropdown, dynamic_wind_plot, align="center"),    # Center align wind dropdown and plot
    sizing_mode='stretch_width',  # Make layout responsive to different screen sizes
    width=1100,  # Set a consistent width for the layout
    height=900   # Adjust height to give more space for plots
)

# Save the grid layout as an HTML file
grid_layout.save('../docs/time_series_plots_with_table.html')

# Display the panel in a notebook or in a web application
grid_layout.show()
